# Lab 03 Tasks

The dataset used for this task consists of records of flights retrieved from the [US Bureau of Transportation Statistics website](https://www.transtats.bts.gov) The data covers the period Q1 and Q2 2016, and includes each flight’s origin, destination, along with other metadata. The raw data is provided as a single CSV file (airstats-2016.csv).

### Task 1

Load the flight record data from the file airstats-2016.csv into a Pandas Data Frame, and apply the following filtering steps to the Data Frame:

1. Only include records where the flight origin and destination where both in the United States.
2. Only include records from the time periods Q1 2016 and Q2 2016.
3. Only include records where the reported distance between the origin and destination was at least 20 miles.

In [2]:
import pandas as pd 

# Load data from CSV file into PD Dataframe
df = pd.read_csv("lab03-data/airstats-2016.csv")
print( "%d flight records" % len(df))
df.head(10)

316274 flight records


,DISTANCE,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_COUNTRY,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_COUNTRY,YEAR,QUARTER,MONTH
0,226,UA,United Air Lines Inc.,12889,LAS,"Las Vegas, NV",US,14908,SNA,"Santa Ana, CA",US,2016,3,9
1,1943,UA,United Air Lines Inc.,12889,LAS,"Las Vegas, NV",US,16271,YYZ,"Toronto, Canada",CA,2016,3,9
2,1814,UA,United Air Lines Inc.,12892,LAX,"Los Angeles, CA",US,12339,IND,"Indianapolis, IN",US,2016,3,9
3,1363,UA,United Air Lines Inc.,12892,LAX,"Los Angeles, CA",US,13198,MCI,"Kansas City, MO",US,2016,3,9
4,1670,UA,United Air Lines Inc.,12892,LAX,"Los Angeles, CA",US,13495,MSY,"New Orleans, LA",US,2016,3,9
5,954,UA,United Air Lines Inc.,12892,LAX,"Los Angeles, CA",US,14747,SEA,"Seattle, WA",US,2016,3,9
6,308,UA,United Air Lines Inc.,12892,LAX,"Los Angeles, CA",US,14831,SJC,"San Jose, CA",US,2016,3,9
7,590,UA,United Air Lines Inc.,12892,LAX,"Los Angeles, CA",US,14869,SLC,"Salt Lake City, UT",US,2016,3,9
8,36,UA,United Air Lines Inc.,12892,LAX,"Los Angeles, CA",US,14908,SNA,"Santa Ana, CA",US,2016,3,9
9,1540,UA,United Air Lines Inc.,12953,LGA,"New York, NY",US,10558,BFF,"Scottsbluff, NE",US,2016,3,9


In [7]:
print("Total Rows: %d" % (len(df)))
print("Number of Rows With Flights with Origin outside of US: %d" % (df['ORIGIN_COUNTRY'].value_counts()['US']))
print("Number of Rows With Flights with Destination outside of US: %d" % (df['DEST_COUNTRY'].value_counts()['US']))

Total Rows: 316274
Number of Rows With Flights with Origin outside of US: 282104
Number of Rows With Flights with Destination outside of US: 280512


In [10]:
df = df[(df['ORIGIN_COUNTRY'] == 'US') & (df['DEST_COUNTRY'] == 'US')]
print("Total Rows: %d" % (len(df)))


Total Rows: 246342


In [12]:
df = df[(df['QUARTER']<3)]
print("Total Rows: %d" % (len(df)))


Total Rows: 120750


In [13]:
df = df[(df['DISTANCE']>=20)]
print("Total Rows: %d" % (len(df)))


Total Rows: 119038


### Task 2

Create an **unweighted directed** network from the Pandas Data Frame.

Use the three letter IATA airport codes for the origin and destination as the node identifiers. Also add the airport’s city name as an attribute for each node.

In [14]:
import networkx as nx

origins = set(df['ORIGIN'].unique())
destinations = set(df['DEST'].unique())
airports = origins.union(destinations)

In [15]:
city_names = {}
for i, row in df.iterrows():
    city_names[row['ORIGIN']] = row['ORIGIN_CITY_NAME']
    city_names[row['DEST']] = row['DEST_CITY_NAME']



In [20]:
#Create Directed Graph
g = nx.DiGraph()
nodes = sorted(list(airports))
for node in nodes: 
    g.add_node(node, city=city_names[node])

In [21]:
for i, row in df.iterrows():
    node1 = row['ORIGIN']
    node2 = row['DEST']
    if node1 == node2:
        continue
    g.add_edge(node1, node2)

### Task 3

Characterise the unweighted directed network from Task 3, looking at:
      
1. How many nodes and edges are in the network?
2. The connectedness of the network (i.e. density and number of components).
3. Identify frequent origin and destination airports in the network (i.e. in-degree and out-degreee)
4. Identify key hub airports in the network (i.e. betweenness)

In [23]:
print("There are %d nodes and %d edges in the network." % (g.number_of_nodes(), g.number_of_edges()))

There are 1043 nodes and 17644 edges in the network.


In [24]:
nx.density(g)

0.01623472818515908

In [26]:
nx.number_strongly_connected_components(g)

92

In [27]:
s_in = pd.Series( dict(g.in_degree()) )
s_out = pd.Series( dict(g.out_degree()) )

In [28]:
s_in.sort_values(ascending=False).head(10)

ORD    200
ATL    187
DEN    184
DFW    176
MSP    175
LAS    155
MEM    152
SDF    151
IAH    151
LAX    147
dtype: int64

In [29]:
s_out.sort_values(ascending=False).head(10)

ORD    205
ATL    193
DEN    187
ANC    183
DFW    175
MSP    171
MEM    162
LAS    162
DTW    152
LAX    151
dtype: int64

In [30]:
s_bet = pd.Series(nx.betweenness_centrality(g))
s_bet.sort_values(ascending=False).head(10)

ANC    0.281623
FAI    0.078011
SEA    0.068236
HPN    0.064631
ORD    0.058346
DEN    0.052816
MSP    0.046166
ADQ    0.035831
DFW    0.033752
ATL    0.033084
dtype: float64

### Task 4

Now create an alternative **weighted directed** network from the Pandas Data Frame.

In [37]:
g2 = nx.DiGraph()
nodes = list(airports)
nodes.sort()
for node in nodes:
    g2.add_node(node, city=city_names[node])

In [38]:
from collections import Counter

freqs = Counter()

for i, rows in df.iterrows():
    node1 = row["ORIGIN"]
    node2 = row["DEST"]
    if node1 == node2:
        continue
    pair = (node1, node2)
    freqs[pair] += 1

In [39]:
for pair in freqs:
    g2.add_edge(pair[0], pair[1], weight=freqs[pair])
print("There are %d nodes and %d edges in the network." % (g2.number_of_nodes(), g2.number_of_edges()))

There are 1043 nodes and 1 edges in the network.


### Task 5

Based on the weighted directed network, identify:
    
1. The most frequent routes in the network.
2. The most frequent origin and destination airports in the network, considering edge weights.

In [ ]:
weights = {}
